In [59]:
# -*- coding: utf-8 -*-

import os,dlib,glob,numpy
from skimage import io

predictor_path = "shape_predictor.dat"
face_rec_model_path = "dlib_face_recognition.dat"
faces_folder_path ='train_images' 

detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor(predictor_path)
facerec = dlib.face_recognition_model_v1(face_rec_model_path)

candidate = []
descriptors = []

for f in glob.glob(os.path.join(faces_folder_path,"*.jpg")):
    print("正在处理: {}".format(f))
    img = io.imread(f)
    candidate.append(f.split('\\')[-1].split('.')[0])
    dets = detector(img, 1)
    for k, d in enumerate(dets): 
        shape = sp(img, d)
        face_descriptor = facerec.compute_face_descriptor(img, shape)
        v = numpy.array(face_descriptor) 
        descriptors.append(v)

print('识别训练完毕！')
readpath=r"./test_images/test10.jpg"
try:
    img = io.imread(readpath)
    dets = detector(img, 1)
except:
    print('输入路径有误，请检查！')

正在处理: train_images/dog10.jpg
正在处理: train_images/dog5.jpg
正在处理: train_images/dog4.jpg
正在处理: train_images/dog6.jpg
正在处理: train_images/dog7.jpg
正在处理: train_images/dog3.jpg
正在处理: train_images/dog2.jpg
正在处理: train_images/dog1.jpg
正在处理: train_images/dog9.jpg
正在处理: train_images/dog8.jpg
识别训练完毕！


In [ ]:
from torch import nn
import torch as t
from torch.nn import functional as F

class ResidualBlock(nn.Module):
    ### 残差单元
    def __init__(self, inchannel, outchannel, stride=1, shortcut=None):
        ### 卷积
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, 3, stride, 1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, 3, 1, 1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.right = shortcut

    def forward(self, x):
        ### 先恒等映射，然后加上卷积后的out再relu
        out = self.left(x)
        residual = x if self.right is None else self.right(x)
        out += residual
        return F.relu(out)


class ResNet34(nn.Module):
    def __init__(self, num_classes=1000):
        super(ResNet34, self).__init__()
        ### 先做 7x7 卷积
        self.pre = nn.Sequential(
            nn.Conv2d(3, 64, 7, 2 ,3, bias=False),      ### 输入 3 通道，输出 64 通道，卷积核7x7，步长2，padding 3
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, 1, 1)       ### inchannel，outchannel，padding
        )
        ### 共32层
        self.layer1 = self._make_layer(64, 128, 3)              ### 3 个 64 通道的残差单元，输出 128通道，共6层
        self.layer2 = self._make_layer(128, 256, 4, stride=2)   ### 4 个 128通道的残差单元，输出 256通道，共8层
        self.layer3 = self._make_layer(256, 512, 6, stride=2)   ### 6 个 256通道的残差单元，输出 512通道，共12层
        self.layer4 = self._make_layer(512, 512, 3, stride=2)   ### 3 个 512通道的残差单元，输出 512通道，共6层
        ### fc，1层
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, inchannel, outchannel, block_num, stride=1):
        ### 1x1 卷积 改变通道数
        shortcut = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, 1, stride, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        layers = []
        layers.append(ResidualBlock(inchannel, outchannel, stride, shortcut))       ### 先来一个残差单元，主要是改变通道数
        ### 再接几个同样的残差单元，通道数不变
        for i in range(1, block_num+1):       ### block_num
            layers.append(ResidualBlock(outchannel, outchannel))
        return nn.Sequential(*layers)

    def forward(self, x):
        ### 第1层
        x = self.pre(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        ### 注意 resnet 最后的池化是把一个 feature map 变成一个特征，故池化野大小等于最后 x 的大小
        x = F.avg_pool2d(x, 2)      ### 这里用的 cifar10 数据集，此时的 x size 为 512x2x2，所以池化野为2

        x = x.view(x.size(0), -1)
        return self.fc(x)


In [ ]:
class ResNet50(object):
    def __init__(self, inputs, num_classes=1000, is_training=True,
                 scope="resnet50"):
        self.inputs =inputs
        self.is_training = is_training
        self.num_classes = num_classes

        with tf.variable_scope(scope):
            # construct the model
            ### 1层
            net = conv2d(inputs, 64, 7, 2, scope="conv1") # -> [batch, 112, 112, 64]
            net = tf.nn.relu(batch_norm(net, is_training=self.is_training, scope="bn1"))
            net = max_pool(net, 3, 2, scope="maxpool1")  # -> [batch, 56, 56, 64]
            ### 每个bottleneck3层， 16 * 3 = 48层
            net = self._block(net, 256, 3, init_stride=1, is_training=self.is_training, scope="block2")           # -> [batch, 56, 56, 256]
            net = self._block(net, 512, 4, is_training=self.is_training, scope="block3")    # -> [batch, 28, 28, 512]
            net = self._block(net, 1024, 6, is_training=self.is_training, scope="block4")   # -> [batch, 14, 14, 1024]
            net = self._block(net, 2048, 3, is_training=self.is_training, scope="block5")   # -> [batch, 7, 7, 2048]
            net = avg_pool(net, 7, scope="avgpool5")                                        # -> [batch, 1, 1, 2048]
            net = tf.squeeze(net, [1, 2], name="SpatialSqueeze")                            # -> [batch, 2048]

            # 1层
            self.logits = fc(net, self.num_classes, "fc6")                                  # -> [batch, num_classes]
            self.predictions = tf.nn.softmax(self.logits)


    def _block(self, x, n_out, n, init_stride=2, is_training=True, scope="block"):
        with tf.variable_scope(scope):
            h_out = n_out // 4
            out = self._bottleneck(x, h_out, n_out, stride=init_stride, is_training=is_training, scope="bottlencek1")
            for i in range(1, n):
                out = self._bottleneck(out, h_out, n_out, is_training=is_training, scope=("bottlencek%s" % (i + 1)))
            return out

    def _bottleneck(self, x, h_out, n_out, stride=None, is_training=True, scope="bottleneck"):
        """ A residual bottleneck unit"""
        n_in = x.get_shape()[-1]
        if stride is None:
            stride = 1 if n_in == n_out else 2

        with tf.variable_scope(scope):
            # 3层
            h = conv2d(x, h_out, 1, stride=stride, scope="conv_1")
            h = batch_norm(h, is_training=is_training, scope="bn_1")
            h = tf.nn.relu(h)
            h = conv2d(h, h_out, 3, stride=1, scope="conv_2")
            h = batch_norm(h, is_training=is_training, scope="bn_2")
            h = tf.nn.relu(h)

            h = conv2d(h, n_out, 1, stride=1, scope="conv_3")
            h = batch_norm(h, is_training=is_training, scope="bn_3")

            if n_in != n_out:
                shortcut = conv2d(x, n_out, 1, stride=stride, scope="conv_4")
                shortcut = batch_norm(shortcut, is_training=is_training, scope="bn_4")
            else:
                shortcut = x
            return tf.nn.relu(shortcut + h)

In [60]:
dist = []
for k, d in enumerate(dets):
    shape = sp(img, d)
    face_descriptor = facerec.compute_face_descriptor(img, shape)
    d_test = numpy.array(face_descriptor) 
    for i in descriptors:
        dist_ = numpy.linalg.norm(i-d_test)
        dist.append(dist_)

In [61]:
c_d = dict(zip(candidate,dist))  
cd_sorted = sorted(c_d.items(), key=lambda d:d[1])

In [62]:
print (readpath,"识别到最有可能是: ",cd_sorted[0][0])
print(c_d)

./test_images/test10.jpg 识别到最有可能是:  train_images/dog10
{'train_images/dog10': 0.714463098821605}


In [3]:
%pip install scikit-image dlib

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
     |████████████████████████████████| 12.1 MB 394 kB/s eta 0:00:01
     |████████████████████████████████| 4.3 MB 131 kB/s eta 0:00:01
     |████████████████████████████████| 148 kB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 233 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.
